In [ ]:
#Auto-reload modules (used to develop functions outside this notebook)
%load_ext autoreload
%autoreload 2

In [ ]:
from RippleNoiseRemoval import RNR
import labrotation.file_handling as fh
import h5py
from time import time

import bokeh.plotting as bpl
import cv2
import glob
import logging
import matplotlib.pyplot as plt
import numpy as np
import os
from labrotation import file_handling as fh
from copy import deepcopy
try:
    cv2.setNumThreads(0)
except():
    pass

import caiman as cm
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import cnmf as cnmf
from caiman.source_extraction.cnmf import params as params
from caiman.utils.utils import download_demo
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour

import json  # for exporting parameters

# for exporting moco data:
from caiman.motion_correction import sliding_window
import cv2

import pandas as pd  # for opening data documentation
import warnings
import uuid  # for generating UUID in case of missing value


from movie_splitting import numpy_to_hdf5
bpl.output_notebook()

In [ ]:
import labrotation.two_photon_session as tps
import datadoc_util as ddoc

In [ ]:
env_dict = dict()
if not os.path.exists("./.env"):
    print(".env does not exist")
else:
    with open("./.env", "r") as f:
        for line in f.readlines():
            l = line.rstrip().split("=")
            env_dict[l[0]] = l[1]
print(env_dict.keys())

In [ ]:
dd = ddoc.DataDocumentation(env_dict["DATA_DOCU_FOLDER"])
dd.loadDataDoc()
if "SERVER_SYMBOL" in env_dict.keys():
    dd.setDataDriveSymbol(env_dict["SERVER_SYMBOL"])
print("data documentation loaded")

In [ ]:
env_dict["MATLAB_2P_FOLDER"]

In [ ]:
for i_row, row in dd.GROUPING_DF.iterrows():
    if not isinstance(row["labview"], float) and not os.path.exists(os.path.join(row["folder"], row["labview"])):  # if does not exist, np.nan, which is float
        print(os.path.join(row["folder"], row["labview"]))

In [ ]:
fold = r["folder"]
fpath_lv = os.path.join(fold, r["labview"])
fpath_lvtime = os.path.join(fold, os.path.splitext(r["labview"])[0] + "time.txt")
fpath_nikmeta = os.path.join(fold, r["nikon_meta"])

In [ ]:
session = tps.TwoPhotonSession.init_and_process(nd2_path = None, nd2_timestamps_path = fpath_nikmeta, labview_path = fpath_lv, labview_timestamps_path = fpath_lvtime, lfp_path = None, matlab_2p_folder = env_dict["MATLAB_2P_FOLDER"])

In [ ]:
# TODO: go over each file; first, without tps.init_and_process, just list the total distance per round max values (so how much the distances add up to)
# Then do the downsampling, and see if it is still the same. Then will need to find a factor for totdist_abs to become same unit.

In [ ]:
with open(fpath_lv, "r") as f:
    l = []
    for line in f.readlines()[:-1]:  # do not deal with incomplete last line
        l.append(list(map(lambda x: float(x), line.split("\t"))))
    l = np.array(l)
    # get first and last row for each round
    i_last_per_round = np.where(l[1:, 0] - l[:-1, 0])[0]
    #i_first = np.concatenate([np.array([0]), i_last[:-1] + 1])  # add first row as beginning, and remove last, incomplete row
    belt_length_vals = l[i_last_per_round, 3]  # column #4 (=index 3) is the distance per round

In [ ]:
dict_recording_belt_lengths = dict()
for i_row, row in dd.GROUPING_DF.iterrows():
    if not isinstance(row["labview"], float):
        fpath_lv = os.path.join(row["folder"], row["labview"])
        try:
            with open(fpath_lv, "r") as f:
                l = []
                for line in f.readlines()[:-1]:  # do not deal with incomplete last line
                    l.append(list(map(lambda x: float(x), line.split("\t"))))
                l = np.array(l)
                # get first and last row for each round
                i_last_per_round = np.where(l[1:, 0] - l[:-1, 0])[0]
                #i_first = np.concatenate([np.array([0]), i_last[:-1] + 1])  # add first row as beginning, and remove last, incomplete row
                belt_length_vals = l[i_last_per_round, 3]  # column #4 (=index 3) is the distance per round
                #if len(belt_length_vals) > 0:
                dict_recording_belt_lengths[fpath_lv] = belt_length_vals
        except IndexError:
            print(f"IndexError for {fpath_lv}")

In [ ]:
dict_recording_belt_lengths2 = dict()
for fname in dict_recording_belt_lengths:
    if len(dict_recording_belt_lengths[fname]) > 0:
        dict_recording_belt_lengths2[fname] = list(dict_recording_belt_lengths[fname])  # convert to list for json compatibility

In [ ]:
out_fpath = os.path.join(env_dict["DOWNLOADS_FOLDER"], "belt_lengths.json")
with open(out_fpath, "w") as f:
    json.dump(dict_recording_belt_lengths2, f, indent=4)

In [ ]:
session.belt_scn_dict["rounds"]

In [ ]:
session.belt_scn_dict

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(session.belt_scn_dict["totdist"])

In [ ]:
plt.plot(session.belt_dict["distance"])

In [ ]:
# totdist and distance are similar, one in tscn_dict, the other in the original... 
#So same normalization should apply. Get it from matlab, apply to totdist and toitdist_abs

In [ ]:
plt.plot(session.belt_scn_dict["speed"])

In [ ]:
session.belt_scn_dict.keys()

In [ ]:
plt.plot(session.belt_scn_dict["distance"] - session.belt_scn_dict["totdist"])

In [ ]:
plt.plot(session.belt_scn_dict["distance"])

In [ ]:
session.belt_scn_dict.keys()

In [ ]:
plt.plot(session.belt_scn_dict["totdist"])

In [ ]:
factor = np.mean(dict_recording_belt_lengths2[fpath_lv])

In [ ]:
plt.plot((1500*session.belt_scn_dict["totdist_abs"]/factor)%1500)
plt.plot(session.belt_scn_dict["distance"])

In [ ]:
factor